In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
import os
import collections
import re 
from imageio import imread, imwrite
from multiprocessing import Pool
import matplotlib.pyplot as plt
import cv2
import gdown
import math

import shutil
import imageio
import time
import torch
#import matplotlib.pyplot as plt
sys.path.insert(0, "../../SemanticStyleGAN")
from models import make_model
from visualize.utils import generate,generate_tensors

## Generate Images

In [ ]:
color_map = {
    0: [0, 0, 0], #Void
    1: [128, 64,128], #Road
    2: [220, 20, 60], # Human
    3: [ 0,  0,142], # Vehicle
    4: [70, 70, 70], # Buildings/Construction
    5: [190,153,153],#Objects
    6: [107,142, 35],# Naturer
    7: [70,130,180], #Sky
}

In [3]:
def tensor2image(tensor):
    images = tensor.cpu().clamp(-1,1).permute(0,2,3,1).numpy()
    images = images * 127.5 + 127.5
    images = images.astype(np.uint8)
    return images

In [110]:
def split_images_segmentation(sample_img,sample_seg):
    image = tensor2image(sample_img)
    seg_dim = sample_seg.size(1)
    sample_seg = torch.argmax(sample_seg, dim=1).detach().cpu().numpy()
    split_masks=[]
    split_images=[]
    for key in range(seg_dim):
        divided_image = np.zeros(image.shape)
        sample_mask = np.zeros((sample_seg.shape[0], sample_seg.shape[1], sample_seg.shape[2], 3), dtype=np.uint8)
        sample_mask[sample_seg==key] = color_map[key]
        divided_image[sample_seg==key]=image[sample_seg==key]
        #divided_image[np.isin(sample_seg,[key,4])]=image[np.isin(sample_seg,[key,4])] #For Several classes
        split_masks.append(sample_mask)
        split_images.append(divided_image)
    return split_masks,split_images

In [115]:
def generate_images(ckpt_loc,truncation_mean=10000,batch=8,truncation=0.7,device="cuda",sample=3,outdir="./data/multiple_d"):
    ckpt = torch.load(ckpt_loc)
    model = make_model(ckpt["args"])
    model.to(device)
    model.eval()
    model.load_state_dict(ckpt["g_ema"])
    mean_latent = model.style(
        torch.randn(truncation_mean, model.style_dim, device=device)
    ).mean(0)
    print("Generating images ...")
    start_time = time.time()
    with torch.no_grad():
        styles = model.style(
            torch.randn(sample, model.style_dim, device=device)
        )
        styles = truncation * styles + (
            1 - truncation
        ) * mean_latent.unsqueeze(0)
        # images, segs = generate(
        #     model, styles, mean_latent=mean_latent, batch_size=batch
        # )

        images, segs,images_tensor,segs_tensor = generate_tensors(
            model, styles, mean_latent=mean_latent, batch_size=batch
        )
        seg_splits,images_splits=split_images_segmentation(images_tensor,segs_tensor)
        for i in range(len(images)):
            imageio.imwrite(f"{outdir}/{str(i).zfill(6)}_img_mul_dis.png", images[i])
            imageio.imwrite(f"{outdir}/{str(i).zfill(6)}_seg_mul_dis.png", segs[i])
            for j in range(len(seg_splits)):
                imageio.imwrite(f"{outdir}/{str(i).zfill(6)}_seg_mul_dis_{j}.png", seg_splits[j][i])
                imageio.imwrite(f"{outdir}/{str(i).zfill(6)}_img_mul_dis_{j}.png", images_splits[j][i])
        return images, segs,images_tensor,segs_tensor
            # if save_latent:
            #     np.save(
            #         f"{outdir}/{str(i).zfill(6)}_latent.npy",
            #         styles[i : i + 1].cpu().numpy(),
            #     )

In [116]:
ckpt_loc="/usr/stud/faragy/storage/user/data/checkpoints/SSG_v2.2/ckpt/380000.pt" #Using the 8 LGs to test
images, segs,images_tensor,segs_tensor=generate_images(ckpt_loc)

Initializing model with arguments:
{'aug': False,
 'base_layers': 2,
 'batch': 4,
 'channel_multiplier': 2,
 'checkpoint_dir': '/usr/stud/faragy/storage/user/data/checkpoints/SSG_v2.2',
 'ckpt': '/usr/stud/faragy/storage/user/data/checkpoints/SSG_v2.2/ckpt/185000.pt',
 'coarse_channel': 512,
 'coarse_size': 64,
 'd_reg_every': 16,
 'dataset': '/usr/stud/faragy/storage/user/data/lmdb_datasets/lmdb_v2.2',
 'depth_layers': 6,
 'detach_texture': False,
 'distributed': False,
 'g_reg_every': 4,
 'inception': '/usr/stud/faragy/storage/user/data/inception_models/inception_v2.2.pkl',
 'iter': 600001,
 'lambda_mask': 100.0,
 'latent': 512,
 'local_channel': 64,
 'local_layers': 10,
 'local_rank': 0,
 'lr': 0.002,
 'min_feat_size': 16,
 'mixing': 0.3,
 'n_gpu': 1,
 'n_mlp': 8,
 'n_sample': 16,
 'num_workers': 8,
 'path_batch_shrink': 2,
 'path_regularize': 0.5,
 'r1_img': 10,
 'r1_seg': 1000,
 'residual_refine': True,
 'save_every': 5000,
 'seg_dim': 8,
 'size': 256,
 'start_iter': 185000,
 'tra

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 230.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 249.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 246.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 234.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 248.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 253.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conver

In [89]:
images.shape

(1, 256, 256, 3)

In [104]:
seg_splits[0][0].shape

(256, 256, 3)

In [106]:
len(seg_splits)

8

In [107]:
#create 1 image for each segmentation class out of result
seg_splits,images_splits=split_images_segmentation(segs_tensor,images_tensor)
outdir = "./data/multiple_d"
for i in range(len(seg_splits)):
    imageio.imwrite(f"{outdir}/{str(0).zfill(6)}_seg_mul_dis_{i}.png", seg_splits[i][0])
    imageio.imwrite(f"{outdir}/{str(0).zfill(6)}_img_mul_dis_{i}.png", images_splits[i][0])


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 186.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 243.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 245.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 127.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


In [97]:
seg_splits[0].shape

(3, 256, 256, 3)

In [98]:
len(seg_splits)

8